<a href="https://colab.research.google.com/github/Knight7561/hf-smol-course/blob/learn/1_instruction_tuning/student_examples/google-research-datasets/mbpp/sft_finetuning_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p>
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
# Install the requirements in Google Colab
!pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login
login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 7.9 MB/s eta 0:00:00


In [3]:
import os
os.environ['WANDB_DISABLED'] = 'true'

In [4]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-eli5_precomputed_top_slice"
finetune_tags = ["smol-course-learn", "module_1","eli5_precomputed_top_slice"]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

# Generate with the base model

Here we will try out the base model which does not have a chat template.

In [5]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a


## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [21]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
ds = load_dataset(path="adamjweintraut/eli5_precomputed_top_slice")

In [22]:
ds

DatasetDict({
    train: Dataset({
        features: ['index', 'q_id', 'question', 'best_answer', 'all_answers', 'num_answers', 'top_answers', 'num_top_answers', 'context', 'orig', 'target'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['index', 'q_id', 'question', 'best_answer', 'all_answers', 'num_answers', 'top_answers', 'num_top_answers', 'context', 'orig', 'target'],
        num_rows: 1250
    })
    validation: Dataset({
        features: ['index', 'q_id', 'question', 'best_answer', 'all_answers', 'num_answers', 'top_answers', 'num_top_answers', 'context', 'orig', 'target'],
        num_rows: 1250
    })
})

In [23]:
# TODO: 🦁 If your dataset is not in a format that TRL can convert to the chat template, you will need to process it. Refer to the [module](../chat_templates.md)
def add_chat_template(row):
  messages=[
        {"role": "user", "content": row['question']},
        {"role": "assistant", "content": row['best_answer']}
    ]
  row['tokenized_message']=tokenizer.apply_chat_template(messages)
  return row

ds=ds.map(add_chat_template)

Map:   0%|          | 0/1250 [00:00<?, ? examples/s]

Map:   0%|          | 0/1250 [00:00<?, ? examples/s]

In [25]:
tokenizer.decode(ds['train'][3]["tokenized_message"])

'<|im_start|>user\nhow do/did pirate radio stations get shut down?<|im_end|>\n<|im_start|>assistant\nProbably using a dish antenna, like satellite TV uses, and triangulation. The antenna picks up signal strongest from where it is pointing at, so they take three readings from three different places and trace the lines back to the source. Assuming that the signal is all they have to go on, this might take several tries, as the method is inaccurate from a long way off. But if they have some other clues that they could use to guess the general location of the broadcaster, then the amount of tries drops to as few as one.<|im_end|>\n'

## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [29]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
    report_to=None
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"]
)

# TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column`

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:160: UserWarning: No `SFTConfig` passed, using `output_dir=tmp_trainer`.
  warnings.warn(f"No `SFTConfig` passed, using `output_dir={output_dir}`.")
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will d

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

KeyError: 'text'

## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [11]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

Step,Training Loss,Validation Loss
50,1.939600,1.948373
100,1.512900,1.875041
150,1.404500,1.920957
200,1.100600,1.983480
250,0.943000,2.082425
300,0.733200,2.260537
350,0.812500,2.180136
400,0.606700,2.343442
450,0.673400,2.330578
500,0.513800,2.500702


In [12]:
# Set our name for the finetune to be saved &/ uploaded to
trainer.push_to_hub(tags=finetune_tags)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1734024111.110e9501ea91.2757.0:   0%|          | 0.00/32.3k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.50k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/knight7561/SmolLM2-FT-mbpp/commit/9240ee670b77a6d570d3a3199484630e4d8ea279', commit_message='End of training', commit_description='', oid='9240ee670b77a6d570d3a3199484630e4d8ea279', pr_url=None, repo_url=RepoUrl('https://huggingface.co/knight7561/SmolLM2-FT-mbpp', endpoint='https://huggingface.co', repo_type='model', repo_id='knight7561/SmolLM2-FT-mbpp'), pr_revision=None, pr_num=None)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>

In [13]:
# Test the fine-tuned model on the same prompt

def test_finetuned_model(prompt):
  # Let's test the base model before training

  # Format with template
  messages = [{"role": "user", "content": prompt}]
  formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

  # Generate response
  inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

  # TODO: use the fine-tuned to model generate a response, just like with the base example.
  output=model.generate(**inputs,max_new_tokens=100)
  print("After training \n")
  print(tokenizer.decode(output[0], skip_special_tokens=True))

In [14]:
test_finetuned_model("why do old cartoons look so jittery?")

After training 

user
Write a function to find the size of the largest subset of a list of numbers so that every pair is divisible.
Write a function to find the length of the longest sub-sequence such that elements are consecutive duplicates of the given list.emptylist
Write a function to find the maximum sum of subsequences of given array with no adjacent elements.emptylist
Write a function to find the length of the longest sub-sequence such that elements are repeated without repeating elements at the end of the sub-sequence.emptylist
Write a function to find the maximum sum of subsequences of given array with no adjacent elements


In [19]:
prompt="Write a function to convert degrees to radians."
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False,add_generation_prompt=True)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: use the fine-tuned to model generate a response, just like with the base example.
output=model.generate(**inputs,max_new_tokens=100)
print("After training \n")
print(tokenizer.decode(output[0], skip_special_tokens=True))

After training 

user
Write a function to convert degrees to radians.
assistant
Write a function to find the length of the shortest
word that contains the letters 'a' and 'b' in the given string.
alphabet
write a function to find the length of the shortest word that contains the letters 'a' and 'b' in the given string.alchemy
write a function to find the length of the shortest word that contains the letters 'a' and 'b' in the given string.alchemy
write a function to find the length of the shortest


In [18]:
formatted_prompt

'<|im_start|>user\nWrite a function to convert degrees to radians.<|im_end|>\n'

## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.